## Description

This Jupyter notebook is part of the "Buckling Analysis of EMULF Delta Floater using the Time Domain Response Reconstruction Method" tutorial. In the previous steps of this tutorial the user is assumed to have exported a structural and a capacity model from GeniE, as well as panel models and wadam input files from HydroD. After these, a few steps are necessary in HydroD and Sima/Bladed (Sima is assumed in this tutorial). Those steps are not part of this tutorial and can be skipped as the resulting input files are provided. 

In this notebook Python is used with OneWorkflow to set up a Sesam Time Domain Response Reconstruction workflow for buckling check according to DNV-RP-C201. This workflow consists of a precomputation step, which is done once, and a reconstruction step, which is done for each design load case (DLC).
 
In the precomputation step, Sesam Core is used to generate unit responses of the structure associated with unit loads, unit motions, and unit waves, accounting for mooring/tower/Morison forces, and radiated and diffracted hydrodynamic pressure, respectively. Sesam Core in this case runs Wadam and Sestra in the background. 
 
In the reconstruction step, and following a coupled analysis (computing forces and motions) in Sima, the local structure response is directly reconstructed using the precomputed unit reponses in combination with the coupled analysis results.
 
As there is no need to generate new loads or perform structural analysis for each new design load case, the method is orders-of-magnitude faster than a standard time domain method.

The capacity model comes as a single JSON file from GeniE, control data is read from a spreadsheet into a dictionary (Python), including file names for coupled analysis result.

<img src="EMULF_Deltafloater.png" alt="Drawing" style="width: 700px;"/>

Results are currently stored in one .csv and one .lis file for each stiffened plate group.
Buckling results are presented as a table of worst usage factor over all time steps and load cases for each stiffened plate group. Note that the first two time steps are ignored, to avoid unrealistic result peaks.

The workflow is split into several cells which quickly summarize the procedure:

- Set up files and folders for the workflow
- Precompute
- Read DLCs from a spreadsheet
- Create tasks for each DLC
- Run the analysis
- Buckling results agreggation over all DLCs



## Set up files and folders for the workflow

Some general workflow properties are setup and the OneWorkflow client is created. The client will be important to manage and execute the workflow.

In [ ]:
from pathlib import Path
import os, sys
from dnv.oneworkflow.utils import *


####### USER INPUT #########

workspaceId = "FOWT_ULS"                    # Set the Id for the workspace
cloudRun = False                            # Set whether to run in the cloud ( in this example set to False)
topsuper = 3                                # superelement type number for the structural model
model_prefix =""                            # prefix for the structural model

#### END of USER INPUT ####


root_folder = os.getcwd()                                   #The root folder is set as the path to the current working directory of the notebook. 
workspacePath = str(Path(root_folder, 'Workspace'))         #The workspace path is set to a subfolder "Workspace" inside the root folder
sys.path.append(os.path.join(root_folder, '..\\..\\PythonModules'))  # Path to PythonModules

Related to the folders defined above, it is assumed in the code that common input files for the workflow are gathered in a folder "CommonFiles" inside the workspace path. If you have not yet, please copy the folder "CommonFiles" from the input files of this tutorial inside the "Workspace" folder.

## Precompute
Sesam Core runs Wadam and Sestra in the background to produce unit-motion response (FD) for
* diffraction (all selected headings) ​(DiR#.SIN)
* radiation and inertia (6 rigid-body dofs) for user-selected frequencies (RaXX#.SIN) 
* hydrostatic pressure and gravity response ​(DiR#.SIN)
* mooring loads and tower forces (UnitNodalR#.SIN) for nodes specified in sima.force
<br/>
<br/>

In [ ]:
import subprocess,os, shutil
from pathlib import Path
import os
import glob
from dnv.sesam.commands import *
from dnv.sesam.commands.executors import execute_command
from dnv.oneworkflow import *

####### USER INPUT #########

precom_execution_folder = r"Precompute\\Execution"    # Folder where Precompute will be run
precom_output_folder =  r"Precompute\\Output"         # Folder where output of precompute is to be stored
precom_input_folder = r"Input\\precompute"            # Folder where input for precompute is stored
precompute_command = "precompute unit-response-json"  # Command line arguments for Sesam Core to run precompute of unit response 

#### END of USER INPUT ####


## Create execution and ouptut folders
execution_folder_path = os.path.join(workspacePath, precom_execution_folder)
output_folder_path = os.path.join(workspacePath, precom_output_folder )
input_folder_path = os.path.join(workspacePath, precom_input_folder)
shutil.rmtree(execution_folder_path, ignore_errors=True)
shutil.rmtree(output_folder_path, ignore_errors=True)
os.makedirs(execution_folder_path, exist_ok=True)
os.makedirs(output_folder_path, exist_ok=True)

precompute_input_file = os.path.join(execution_folder_path, "precompute_input.json")


## Create a python dictionary then replace the precompute template file with the strings from the dictionary
## All paths are relative to the workspace folder
precompute_parameters= {'inputpath': precom_input_folder,                   
                        'outputpath': precom_output_folder,
                        'executepath': precom_execution_folder,
                        'simaforce': precom_input_folder + r"\\sima.force",
                        'femfile' : precom_input_folder + r"\\" + model_prefix + "T" + str(topsuper) + ".FEM"}

precompute_input_command = CreateInputFileFromFileTemplateCommand(               # Create a command to generate the precompute_input.json by taking the precompute_input_template.json and replacing the strings with parameters for the DLC
        template_input_file  = "precompute_input_template.json",
        input_filename  = precompute_input_file,
        parameters= precompute_parameters,
        working_dir = input_folder_path
    )
execute_command(precompute_input_command)                                        # Execute the command     

## Run precompute unit response in Sesam Core
print("Running Sesam Core in folder " + execution_folder_path)
score_command = SesamCoreCommand(working_dir=workspacePath)                      # create a new Sesam Core command
score_command.arguments = precompute_command + ' -i ' + precompute_input_file    # add command line arguments to the command (run precompute)
execute_command(score_command)                                                   # execute the command to run precompute

## Check if precomputation was successful
This is a rudimentary check which succeeds if it finds the expected output files

In [ ]:
print('Checking if Sesam Core precompute command has been executed successfully')
error = False

# There should be 8 .sin files + SCORE.MLG in the output folder
if (len(glob.glob(os.path.join(output_folder_path, "*.sin"))) != 8):
    print("Precompute command failed - not all SIN files have been created")
    error = True
if (not os.path.exists(os.path.join(output_folder_path, "SCORE.MLG"))):
    print("Precompute command failed - cannot find SCORE.MLG file in the output folder")
    error = True

if not error:
    print("Precompute command executed successfully")

## Read DLCs from a spreadsheet

In the code below the name of the spreadsheet is declared as well as the capacity model file.
In the parameter mapping an association of parameters is done which will be used for assigning each entry in the spreadsheet a corresponding variable. 

In [ ]:
import pandas as pd

####### USER INPUT #########

dlc_sheet = "DLC_input.xlsx"         # The name of the spreadsheet with the parameters of each DLC
capacity = "EMULF_ULS_All.json"      # Name of the capacity model

#### END of USER INPUT ####

## In the definition below the parameter mapping is defined which associates each column in the excel sheet with a corresponding variable in the code. 
## If you use an excel sheet with a different structure, this is where the workflow can be adjusted to interpret that sheet:
parameter_mapping = {'Group': "group",
				'LoadCase': "loadcase",
				"SimaFilesFolder": "sima_files_folder",
                "TimeStep": "timestep",
				"Depth": "depth",
				"StartBuckling": "BUCKLINGSTART",
				"StopBuckling": "BUCKLINGEND",
				                }

input_data = pd.read_excel(os.path.join(workspacePath, dlc_sheet), index_col=1) # in here the excel sheet is read
input_data

fixed_parameters_all_load_cases = {
    'mor_sel': topsuper,                                  # superelement number for the morison model, in this case the same as the structural model
    'prefix': model_prefix,                               # prefix for the morison (and structural) model
    'cap_mod': capacity,                                  # name of the capacity model, for the input.json
    'modelf' : model_prefix+"T"+str(topsuper) + ".FEM",   # name of the model file, for the input.json
    'loadf' :  model_prefix+"L"+str(topsuper) + ".FEM"    # name of the load file, for the input.json 
    }


## Create tasks for each DLC

In the code below the tasks are created for each analysis and for each DLC. The user may select to skip part of the workflow in the USER INPUT section, in that case make sure all necessary files for the following analysis are available. Moreover, to avoid deleting these files at the beginning of the run then declare `clean = False`.

- The template files are populated with the proper values for each DLC using the [`CreateInputFileFromFileTemplateCommand`](https://myworkspace.dnv.com/download/public/sesam/workflow/docs/latest/source/dnv.oneworkflow.html#dnv.oneworkflow.create_input_file_from_file_template_command.CreateInputFileFromFileTemplateCommand) (the same method was used previously to generate the input fiel for the precompute step)

In [ ]:
import os
import shutil
from dnv.sesam.commands import *
from dnv.oneworkflow import *

####### USER INPUT #########

# Set True for the workflow steps to be run, and False for the steps to be skipped:

run_sestra_template_command = True        # Define whether to update sestra input files with the latest variables
run_sesam_core_template_command = True    # Define whether to update sesam core input files with the latest variables
run_sesam_core = True                     # Define whether to run sestra and sesam core
run_sesam_core_input_command = True

clean = True                              # Define whether to clean (True) or keep (False) files (input and ouptut) from a previous run, if skipping any anlysis step this should be set to false.


#### END of USER INPUT ####

lc_path = os.path.join(workspacePath,"LoadCases")

## Create loadcases folders
if clean:
    shutil.rmtree(lc_path, ignore_errors=True)

workflow_sequence = []
display(input_data)

## Loop over all DLCs is run to get the parameters for each DLC, creating the DLC folders and setup the tasks.
for casename, case in input_data.iterrows():        
    tasks = []
    casedict = case.to_dict()
    input_parameters = {}
    
    for key, value in case.items():
        input_parameters[parameter_mapping[key]] = str(value)

    get_folder=os.path.join(workspacePath, input_parameters["sima_files_folder"])     # Define the path to the folder where the Sima files for this DLC are stored
    DLC_folder=os.path.join(lc_path, case.name)                                        # Define the path of the DLC folder, where the analysis will be run
   
    if not os.path.exists(DLC_folder):          
        os.makedirs(DLC_folder)                                                       # Create the DLC folder, if it does not exist already
    
    shutil.copytree(get_folder, DLC_folder, dirs_exist_ok=True)                       # Copy files from the Sima files folder to the DLC folder

    input_parameters = input_parameters | fixed_parameters_all_load_cases
    print("The following parameters are used for load case: " + casename)
    print(json.dumps(input_parameters, indent=4, sort_keys=True))
 
    sesam_core_template_command = CreateInputFileFromFileTemplateCommand(             # Create the SesamCore_buckling.jnl by taking the SesamCore_buckling_template.jnl and replacing the strings with parameters for the DLC
        template_input_file  = "SesamCore_buckling_template.jnl",
        input_filename  = "SesamCore_buckling.jnl",
        parameters= input_parameters
    )
    if run_sesam_core_template_command:
        tasks.append(sesam_core_template_command)
    
    sesam_core_input_command = CreateInputFileFromFileTemplateCommand(                # Create the SesamCore_buckling.jnl by taking the SesamCore_buckling_template.jnl and replacing the strings with parameters for the DLC
        template_input_file  = "input_template.json",
        input_filename  = "input.json",
        parameters= input_parameters
    )

    if run_sesam_core_input_command:                                                  # Create the input.json by taking the inpu_template.json and replacing the strings with parameters for the DLC
        tasks.append(sesam_core_input_command)

    if run_sesam_core:
        score_command = SesamCoreCommand(working_dir=workspacePath)    
        score_command.arguments = "uls -i input.json -rec -pt 8"
        tasks.append(score_command)
        
    workflow_sequence.append(CommandInfo(commands=tasks,load_case_foldername=casename)) # Add the generated tasks to the workflow sequence.

## Run the analysis

In the code below we run the analysis using [`run_managed_commands_in_parallel_async`](https://myworkspace.dnv.com/download/public/sesam/workflow/docs/latest/source/dnv.oneworkflow.utils.html#dnv.oneworkflow.utils.workflow_commands_runners_parallel.run_managed_commands_in_parallel_async) with the previously created workflow client and the commands setup in the previous cell. This implies that analysis of each DLC will happen in parallel, as opposed to in sequence. We set `log_job=false` to get a more restricted log, but it can be set to `True` for more detailed logs if wanted. We set `enable_common_files_copy_to_load_cases=True` to copy files from the "CommonFiles" folder into every single DLC folder. 

Subsequently, the `analysisStatusChecker` is used to check whether the analysis was successful by checking Sesam Core output files. 

In [ ]:
## Run the analysis
workflow_client = one_workflow_client(workspace_id = workspaceId, cloud_run = cloudRun, workspace_path = workspacePath, inplace_execution = True)
print("Running commands in parallel")
await run_managed_commands_in_parallel_async(
             client=workflow_client,
             commands_info=workflow_sequence,
             log_job=False,
             enable_common_files_copy_to_load_cases=True,
 )

## Check if analysis is successful 

local_result_path = Path(workspacePath, workflow_client.results_directory)
prev_dir = os.getcwd()

try:
    os.chdir(local_result_path)
    import analysisStatusChecker
    analysisStatusChecker.checkSesamCoreStatus()
except:
    print('Error during analysis status check, Sesam Core may not have run successfully. Check the log files.')
    print('Please see SCORE.MLG for more information.')
finally:
    os.chdir(prev_dir)


## Buckling results aggregation over all DLCs
Print maximum usage factor and criterion for worst stiffener on each plate group, over all load cases. 

Please note that unrealistic code check results might happen as this tutorial is work in progress.

In [ ]:
import os
from pathlib import Path
import glob
import pandas as pd

from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
current_date = now.strftime("%d/%m/%Y")

print("Current Time =", current_time)
print("Current Date =", current_date)

wd = os.getcwd()

try:
    panels_and_cases = pd.DataFrame()
    panel_names = set()
    for loadcase_folder_name, _ in input_data.iterrows():
        result_folder = os.path.join(workspacePath, workflow_client.results_directory, loadcase_folder_name)

        os.chdir(result_folder)

        # loop over all panel result files (csv)
        # Set threshold value for removing the first few result cases to omit extreme initial condition results

        for file in glob.glob('SesamCore_buckling_*Panel*.csv'):
            gen = pd.read_csv(file, dtype={"math score":int}, chunksize=10000000)
            panel = pd.concat((x.query("`Result case id` >= 3") for x in gen), ignore_index=True)
            panel['Plate field'] = Path(file).stem
            panel['LoadCase'] = loadcase_folder_name
            if not panel.empty:
                panels_and_cases = pd.concat([panels_and_cases,panel])
                panel_names.add(Path(file).stem)

except:
    print("Error accumulating results, please inspect analysis resultes manually.") 
finally:
    os.chdir(wd)

## Aggregating all results in a single table
results_table = pd.DataFrame(columns=['Stiffener name', 'Plate field', 'UfMax', 'UfMax criterion', 'LoadCase', 'Time'])
for panel_name in panel_names:
    panel_results = panels_and_cases.query('`Plate field` == @panel_name')
    panel_results.reset_index(inplace=True)
    if not panel_results.empty:
        results_table.loc[len(results_table.index)] = dict(panel_results.iloc[panel_results['UfMax'].idxmax()])
        
## Sorting by UfMax
results_table.sort_values(by='UfMax', ascending=False, inplace=True)

print(f"Maximum Uf for each panel, taken over all load cases and result cases.")
print(f"Note that the first two time steps (usually extreme results) have been filtered out to avoid reporting unrealistic peak results.")

# Adjust the display settings to show all rows
pd.options.display.max_rows = 800

## Pandas provides many different ways to visualize the data. Here we just dump the results to the output
display(results_table)

## Pandas DataFrames can be exported to a number of different output formats, such as Excel, CSV, json, XML, HTML, Parquet, etc.
## Uncomment the lines below to experiment with different output formats
# results_table.to_excel(os.path.join(workspacePath, 'results.xlsx'), index=False)
# results_table.to_html(os.path.join(workspacePath, 'results.html'), index=False)

